In [1]:
import torch
from torch_geometric import nn, datasets
import pytorch_lightning as L
import torch.nn.functional as F
from torcheval import metrics

import os
from pathlib import Path

In [2]:
dataset = datasets.Planetoid("./datasets", "Cora")

Processing...
Done!


In [3]:
print(f"Name: {dataset.name}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of edge features: {dataset.num_edge_features}")
print(f"Number of node features: {dataset.num_node_features:,}")

Name: Cora
Number of classes: 7
Number of edge features: 0
Number of node features: 1,433


In [4]:
class GCNModel(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        num_layers: int,
        dropout: float,
    ):
        super(GCNModel, self).__init__()
        self.gcn = torch.nn.ModuleList()
        for i in range(num_layers):
            self.gcn.append(
                nn.GraphConv(
                    in_channels if i == 0 else hidden_channels, hidden_channels
                )
        )
        self.act = torch.nn.ReLU()

        self.dropout = torch.nn.Dropout(dropout)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)
        self.softmax = torch.nn.Softmax(-1)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        h = x
        for layer in self.gcn:
            h = layer(h, edge_index)
            h = self.act(h)

        h = self.dropout(h)
        h = self.lin(h)
        h = self.softmax(h)

        return h

In [5]:
class GCNTrainer(L.LightningModule):
    def __init__(
        self,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        num_layers: int,
        lr: float,
        dropout: float,
    ):
        super().__init__()
        self.model = GCNModel(
            in_channels,
            hidden_channels,
            out_channels,
            num_layers,
            dropout,
        )
        self.lr = lr
        self.criterion = torch.nn.CrossEntropyLoss()
        self.save_hyperparameters()

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor):
        return self.model(x, edge_index)

    def training_step(self, sample: torch.Tensor):
        acc = metrics.MulticlassAccuracy()
        x, y, edge_index = sample.x, F.one_hot(sample.y), sample.edge_index
        pred = self.model(x, edge_index)
        loss = self.criterion(y[sample.train_mask].float(), pred[sample.train_mask])
        acc.update(sample.y[sample.train_mask].float(), torch.argmax(pred[sample.train_mask], -1))
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_acc", acc.compute(), prog_bar=True, logger=True)
        return loss

    def validation_step(self, sample: torch.Tensor):
        acc = metrics.MulticlassAccuracy()
        x, y, edge_index = sample.x, F.one_hot(sample.y), sample.edge_index
        pred = self.model(x, edge_index)
        loss = self.criterion(y[sample.val_mask].float(), pred[sample.val_mask])
        acc.update(sample.y[sample.val_mask].float(), torch.argmax(pred[sample.val_mask], -1))
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_acc", acc.compute(), prog_bar=True, logger=True)
        return loss

    def test_step(self, sample: torch.Tensor):
        acc = metrics.MulticlassAccuracy()
        x, y, edge_index = sample.x, F.one_hot(sample.y), sample.edge_index
        pred = self.model(x, edge_index)
        loss = self.criterion(y[sample.test_mask].float(), pred[sample.test_mask])
        acc.update(sample.y[sample.test_mask].float(), torch.argmax(pred[sample.test_mask], -1))
        self.log("test_loss",loss, prog_bar=True, logger=True)
        self.log("test_acc", acc.compute(), prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [6]:
early_stop = L.callbacks.EarlyStopping(monitor="val_loss", mode="min")
timer = L.callbacks.Timer(duration="00:12:00:00")
model_checkpoint = L.callbacks.ModelCheckpoint("./lightning_logs")
callbacks = [early_stop, timer]

In [7]:
trainer_module = GCNTrainer(dataset.num_features, 32, dataset.num_classes, 2, 1e-3, 0.2)
trainer = L.Trainer(callbacks=callbacks, accelerator="gpu", precision=16)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning_fabric\connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
trainer.fit(trainer_module, dataset, dataset)

Missing logger folder: d:\Experiments\Practice\Practices\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | GCNModel         | 94.1 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
94.1 K    Trainable params
0         Non-trainable params
94.1 K    Total params
0.376     Total estimated model params size (MB)


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2708. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 78: 100%|██████████| 1/1 [00:00<00:00, 12.60it/s, v_num=0, train_loss=1.190, train_acc=0.993, val_loss=1.480, val_acc=0.762]


In [9]:
print(f"{timer.time_elapsed('train'):.3f}s")
print(f"{timer.time_elapsed('validate'):.3f}s")
print(f"{timer.time_elapsed('test'):.3f}s")

6.531s
0.032s
0.000s


In [10]:
paths = sorted(Path("./lightning_logs").iterdir(), key=os.path.getmtime, reverse=True)

ckpt_path = os.path.join(paths[0], "checkpoints")
ckpt_file = os.listdir(ckpt_path)[0]
ckpt_full_path = os.path.join(ckpt_path, ckpt_file)

In [11]:
ckpt_full_path

'lightning_logs\\version_0\\checkpoints\\epoch=78-step=79.ckpt'

In [12]:
model = GCNTrainer.load_from_checkpoint(ckpt_full_path)

In [13]:
trainer.test(model, dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 30.76it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7519999742507935
        test_loss           1.4835654497146606
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.4835654497146606, 'test_acc': 0.7519999742507935}]